<a href="https://colab.research.google.com/github/ZabalaGaston/HPC/blob/main/Zabala_Gaston_ejercicio_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Crea una seccion paralela y le ponemos la directiva for indicando que es un ciclo for,
entonces para cada valor de i, va estar creando un hilo que resuelve una parte del algoritmo

Va a ir tomando tantos valores como el tamaño del vector.

Por defecto, openmp sabe que vamos a iterar por el valor i, porque es el valor que se toma para iterar.

Se crean la cantidad de hilos que le decimos, y mientras dure o haya valores para procesar, los hilos van a seguir vivos.

Pendiente: calcular el promedio en C
**Pasarle los parámetros por phyton**


In [29]:
#OpenMP
code = """

#include <iostream>
#include <vector>
#include <cstdlib>
#include  <sys/time.h>
#include <omp.h> //Cabecera OpenMP


//------------------------------------------------
// Macros para medir el tiempo

static double dHashTiempoHistory[3];
static struct timeval tv;

#define TIEMPO_INI( h )      \
  gettimeofday(&tv, NULL);    \
  dHashTiempoHistory[ h ] = tv.tv_sec + tv.tv_usec/1000000.0;
  
#define TIEMPO_FIN( h )      \
  gettimeofday(&tv, NULL);    \
  dHashTiempoHistory[ h ] = ((tv.tv_sec + tv.tv_usec/1000000.0) - dHashTiempoHistory[ h ]) * 1000; //

#define TIEMPO_GET( h ) dHashTiempoHistory[ h ]

#define HTH_TOTAL     1
#define HTH_ACEL_SEC  2
#define HTH_ACEL_OMP  3

//---------------------------------------------------------

int main(int argc, char*argv[])
{
  int i,c;
  TIEMPO_INI ( HTH_TOTAL )

  //Leo los parametros.
  if (argc != 4 )
  {
    std::cerr<<"Error al ingresar los parámetros" <<std::endl;
    exit( - 1 );
   }

   int time       =  atoi( argv[1] );
   int cantidad_N =  atoi( argv[2] );
   int ciclos     =  atoi( argv[3] );

   //-----------------------------------------------------
   // Defino la memoria de los vectores para la velocidad inicial y final

   std:: vector<double>   VI ( cantidad_N );
   std:: vector<double>   VF ( cantidad_N );
   std:: vector<double>   AC ( cantidad_N );

   for (int i = 0; i < cantidad_N; i++)
   {
      VI[i] = 44*0.1; //obtener función en c para calcular numeros aleatorios
      VF[i] = 99*0.1; //
   }

  //------------------------------------------
  // Realizo el cálculo de la aceleración en forma secuencial

  TIEMPO_INI ( HTH_ACEL_SEC )

  for(c=0; c<ciclos; c++)
  {
    for(i=0; i< cantidad_N; i++)
    {
      AC[i] = (VF[i] - VI[i]) / time;
    }
  }

  TIEMPO_FIN ( HTH_ACEL_SEC )

  // ---------------------------------------------
  // Realizo el cálculo de la aceleración con OpenMP

  TIEMPO_INI ( HTH_ACEL_OMP )

  for(c=0; c<ciclos; c++)
  {
    #pragma omp parallel for
    for(i=0; i< cantidad_N; i++)
    {
      AC[i] = (VF[i] - VI[i]) / time;
    }
  }

  TIEMPO_FIN ( HTH_ACEL_OMP )

  //------------------------------------------------

  std::cout<<"Valores Reales  : " <<std::endl;
  std::cout<<"Tiempo total    : " <<TIEMPO_GET( HTH_TOTAL )<<  " [ms]"<<std::endl;
  std::cout<<"Tiempo Acel sec : " <<TIEMPO_GET( HTH_ACEL_SEC)<< " [ms]"<<std::endl;
  std::cout<<"Tiempo Acel Omp : " <<TIEMPO_GET( HTH_ACEL_OMP)<< " [ms]"<<std::endl;
  std::cout<<std::endl;  
}
"""

text_file = open("code_acel.cpp","w")
text_file.write(code)
text_file.close()



##2.1 Compilación de código C (cálculo de aceleración)

In [30]:
!g++ -o acel -fopenmp code_acel.cpp

#2.2 Ejecucion Cálculo de aceleración

In [31]:
%env OMP_NUM_THREADS=2
!./acel 10 40000  20

env: OMP_NUM_THREADS=2
Valores Reales  : 
Tiempo total    : 1.60582e+09 [ms]
Tiempo Acel sec : 9.90415 [ms]
Tiempo Acel Omp : 7.25484 [ms]

